In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
# regular expression
import re
# numerical analysis
import numpy as np

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.worldometers.info/coronavirus/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[0]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
# df_bs.drop('S. No.', axis=1, inplace=True)

# there are 36 states+UT in India
df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"2,617,476","+7,186","151,072",+697,"1,156,401","+4,057","1,310,003","19,305",,,,,,North America,\n,,
1,,\nSouth America\n,"1,629,067",+814,"66,942",+18,"902,856","+16,853","659,269","12,308",,,,,,South America,\n,,
2,,\nEurope\n,"2,277,060","+8,924","185,432",+200,"1,230,209","+11,076","861,419","6,414",,,,,,Europe,\n,,
3,,\nAsia\n,"1,782,937","+7,635","45,623",+168,"1,117,984","+3,460","619,330","16,074",,,,,,Asia,\n,,
4,,\nAfrica\n,"279,452",+775,"7,456",+6,"130,325","+1,978","141,671",506,,,,,,Africa,\n,,


In [6]:
# drop unwanted rows
df_bs = df_bs.iloc[8:, :-3].reset_index(drop=True)

# drop unwanted columns
df_bs = df_bs.drop('#', axis=1)

# first few rows
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"2,263,749",+98,"120,688",,"931,076",+82,"1,211,985","16,503","6,840",365,"26,724,848","80,755","330,938,757",North America
1,Brazil,"983,359",,"47,869",,"520,360","+16,853","415,130","8,318","4,627",225,"2,344,437","11,032","212,508,714",South America
2,Russia,"569,063","+7,972","7,841",+181,"324,406","+10,443","236,816","2,300","3,899",54,"16,321,964","111,846","145,932,574",Europe
3,India,"381,485",+394,"12,605",+1,"205,183",+1,"163,697","8,944",277,9,"6,426,627","4,659","1,379,530,022",Asia
4,UK,"300,469",,"42,288",,N/A,N/A,N/A,379,"4,427",623,"7,259,555","106,956","67,874,383",Europe


In [7]:
# rename columns
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

# rearrange and subselect columns
df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"330,938,757","2,263,749",+98,"120,688",,"931,076",+82,"1,211,985","16,503","6,840",365,"26,724,848","80,755"
1,Brazil,South America,"212,508,714","983,359",,"47,869",,"520,360","+16,853","415,130","8,318","4,627",225,"2,344,437","11,032"
2,Russia,Europe,"145,932,574","569,063","+7,972","7,841",+181,"324,406","+10,443","236,816","2,300","3,899",54,"16,321,964","111,846"
3,India,Asia,"1,379,530,022","381,485",+394,"12,605",+1,"205,183",+1,"163,697","8,944",277,9,"6,426,627","4,659"
4,UK,Europe,"67,874,383","300,469",,"42,288",,N/A,N/A,N/A,379,"4,427",623,"7,259,555","106,956"


In [8]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [9]:
# addding 'WHO Region' column
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)

# countries with missing "WHO Region"
df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['CAR', 'Mayotte', 'French Guiana', 'Diamond Princess',
       'Channel Islands', 'Réunion', 'Isle of Man', 'Martinique',
       'Cayman Islands', 'Faeroe Islands', 'Gibraltar', 'Guadeloupe',
       'Brunei ', 'Aruba', 'Sint Maarten', 'French Polynesia', 'Macao',
       'Saint Martin', 'St. Vincent Grenadines', 'Curaçao',
       'New Caledonia', 'Falkland Islands', 'Turks and Caicos',
       'Montserrat', 'MS Zaandam', 'British Virgin Islands'], dtype=object)

In [10]:
# fix data
for col in df_bs.columns[2:]:
    # replace comma with empty string
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_bs = df_bs.replace('', np.nan)

# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,330938757,2263749,98,120688,NaN,931076,82,1211985,16503,6840,365,26724848,80755,Americas
1,Brazil,South America,212508714,983359,NaN,47869,NaN,520360,16853,415130,8318,4627,225,2344437,11032,Americas
2,Russia,Europe,145932574,569063,7972,7841,181,324406,10443,236816,2300,3899,54,16321964,111846,Europe
3,India,Asia,1379530022,381485,394,12605,1,205183,1,163697,8944,277,9,6426627,4659,South-EastAsia
4,UK,Europe,67874383,300469,NaN,42288,NaN,NaN,NaN,NaN,379,4427,623,7259555,106956,Europe


In [11]:
# save as .csv file
df_bs.to_csv('worldometer_data.csv', index=False)